# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part 4: Training our End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a simple - but fairly effective - logistic regression model.  More generally, however, Snorkel plugs in with many ML libraries including [TensorFlow](https://www.tensorflow.org/), making it easy to use almost any state-of-the-art model as the end extractor!

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import re
import numpy as np

# Connect to the database backend and initalize a Snorkel session
from lib.init import *
from snorkel.models import candidate_subclass
from snorkel.annotations import load_gold_labels

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We repeat our definition of the `Spouse` `Candidate` subclass, and load the test set:

# 1 Training a `SparseLogReg` Discriminative Model
We use the training marginals to train a discriminative model that classifies each `Candidate` as a true or false mention. We'll use a random hyperparameter search, evaluated on the development set labels, to find the best hyperparameters for our model. To run a hyperparameter search, we need labels for a development set. If they aren't already available, we can manually create labels using the Viewer.

## Feature Extraction
Instead of using a deep learning approach to start, let's look at a standard sparse logistic regression model. First, we need to extract out features. This can take a while, but we only have to do it once!

In [6]:
from snorkel.annotations import FeatureAnnotator
from lib.features import hybrid_span_mention_ftrs

featurizer = FeatureAnnotator()

In [7]:
F_train = featurizer.load_matrix(session, split=0)
F_dev = featurizer.load_matrix(session, split=1)
F_test = featurizer.load_matrix(session, split=2)

# if F_train.size == 0:    
#     %time F_train = featurizer.apply(split=0, parallelism=2, annotation_generator=hybrid_span_mention_ftrs)
# if F_dev.size == 0:     
#     %time F_dev  = featurizer.apply_existing(split=1, parallelism=2, annotation_generator=hybrid_span_mention_ftrs)
# if F_test.size == 0:
#     %time F_test = featurizer.apply_existing(split=2, parallelism=2, annotation_generator=hybrid_span_mention_ftrs)




First, reload the training marginals:

In [8]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, F_train, split=0)

In [ ]:
from snorkel.learning import SparseLogisticRegression
disc_model = SparseLogisticRegression()

The following code performs model selection by tuning our learning algorithm's hyperparamters.

In [ ]:
from snorkel.learning.utils import MentionScorer
from snorkel.learning import RandomSearch, ListParameter, RangeParameter

# Searching over learning rate
rate_param        = RangeParameter('lr', 1e-6, 1e-2, step=1, log_base=10)
l1_param          = RangeParameter('l1_penalty', 1e-6, 1e-2, step=1, log_base=10)
l2_param          = RangeParameter('l2_penalty', 1e-6, 1e-2, step=1, log_base=10)
batch_size_param  = ListParameter('batch_size', [32, 64, 128])
b_param           = ListParameter('b', [0.5, 0.6, 0.7])
balance_param     = ListParameter('rebalance', [0.0, 0.3, 0.5])

param_grid = [rate_param, l1_param, l2_param, batch_size_param, b_param, balance_param]

np.random.seed(1701)
searcher = RandomSearch(SparseLogisticRegression, param_grid, F_train,
                        Y_train=train_marginals, n=5, n_threads=1)

logreg, run_stats = searcher.fit(F_dev, L_gold_dev, n_epochs=2000, print_freq=250, n_threads=2)
print run_stats

Initialized RandomSearch search of size 5. Search space size = 3375.
Loading data...
Launching jobs...
[SparseLogisticRegression] Training model
[SparseLogisticRegression] n_train=8757  #epochs=2000  batch size=64
[SparseLogisticRegression] Training model
[SparseLogisticRegression] n_train=17214  #epochs=2000  batch size=32
[SparseLogisticRegression] Epoch 0 (1.43s)	Average loss=1.043090
[SparseLogisticRegression] Epoch 0 (3.34s)	Average loss=2.560153
[SparseLogisticRegression] Epoch 250 (352.39s)	Average loss=4.345488
[SparseLogisticRegression] Epoch 500 (710.75s)	Average loss=12.152730
[SparseLogisticRegression] Epoch 250 (832.17s)	Average loss=2.388735
[SparseLogisticRegression] Epoch 750 (1062.99s)	Average loss=24.711330
[SparseLogisticRegression] Epoch 1000 (1422.07s)	Average loss=42.080006
[SparseLogisticRegression] Epoch 500 (1661.71s)	Average loss=2.405527
[SparseLogisticRegression] Epoch 1250 (1773.06s)	Average loss=64.184975
[SparseLogisticRegression] Epoch 1500 (2131.31s)	Av

In [ ]:
from snorkel.annotations import load_gold_labels

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev.shape

In [ ]:
np.random.seed(1701)
searcher.fit(F_dev, L_gold_dev, n_epochs=50, rebalance=0.5, print_freq=25)

## Examining Features
Extracting features allows us to inspect and interperet our learned weights 

In [ ]:
from lib.scoring import *
print_top_k_features(session, logreg, F_train, top_k=25)

## Evaluate on Test Data

In [ ]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

In [ ]:
_, _, _, _ = disc_model.score(session, F_test, L_gold_test)